In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import torch.nn as nn
from torch import optim
from sklearn.utils import class_weight
from scipy.stats import *
from torch.autograd import Variable
from tqdm.notebook import tqdm

from save_data import save_data
from sigma import sigma
from get_test_stats import get_test_stats
from size_out import *
from test import test
from train import train
from parameters import *
from read_data import read_data
from preproc_data import *
from label_data import label_data
from split_data import split_data
from detection import *
from config_reader import Config
from pathlib import Path

cfg = Config()
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
cyclone_events, data = read_data(path='../data')
# Consider renaming first variable
cyclone_events_data, data, metrics = preproc_data(cyclone_events, metrics=data)
labels, events = label_data(cyclone_events_data.ce2)
train_data, test_data, labels_train, labels_test, train_id, test_id = split_data(data, labels, ratio=0.8)

In [51]:
desc = "Kept batch_size=24. Fixed b, c, and taught a and w."
filename = "Cyclone_bs24_de_fixed_c.ipynb"
network_name = filename.split('.')[0]
comment = "Slight improvement on train, but degradation on test."

In [52]:
class MyLayer1(nn.Module):
    
    def __init__(self, d, e):
        super().__init__()  
        self.d = d
        self.e = e
        
    def forward(self, x):
        # x * d
        x = torch.mul(x, self.d)
        # x * d - e
        x = torch.sub(x, self.e)
        return x

class MyLayer2(nn.Module):
    def __init__(self, c):
        super().__init__()
        self.c = c
        
    def forward(self, x):
        x = torch.sum(x, dim=(2, 3))
        x = x.reshape(x.shape[0])
        x = torch.sub(x, self.c)
        return x

class SimpleNN(nn.Module):
    def __init__(self, d, e, c):
        super(SimpleNN, self).__init__()
        self.d = nn.Parameter(d, requires_grad=True)  
        self.e = nn.Parameter(e, requires_grad=True)
        self.c = c
        
        self.auc_train = nn.Parameter(torch.tensor(0, dtype=torch.float64), requires_grad=False)
        self.auc_test = nn.Parameter(torch.tensor(0, dtype=torch.float64), requires_grad=False)

        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.layer1 = MyLayer1(self.d, self.e)
        self.sigmoid = nn.Sigmoid()
        self.layer2 = MyLayer2(self.c)
    
    def forward(self, x):
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.sigmoid(x)
        x = self.layer2(x)
        return x

$$\sigma(\sum(\sigma(d * x - e)) - c)$$

In [53]:
num_epochs = 80
batch_size = 24

d = torch.empty(size=(), dtype=torch.float)
e = torch.empty(size=(), dtype=torch.float)
c = torch.empty(size=(), dtype=torch.float)

nn.init.constant_(d, val=25)
nn.init.constant_(e, val=50)
nn.init.constant_(c, val=2)

cnn = SimpleNN(d, e, c)

model_path = '../pretrained_models/models/' + cfg.mode + '/' + cfg.metric + '/'
model_name = filename.split('.')[0] + '.pt'
md_path = Path(model_path + model_name)

if not md_path.exists():
    Path(model_path).mkdir(parents=True, exist_ok=True)
    torch.save(cnn, md_path)

statedict_path = '../pretrained_models/statedicts/' + cfg.mode + '/' + cfg.metric + '/'
statedict_name = filename.split('.')[0] + '.pt'
sd_path = Path(statedict_path + statedict_name)

y = torch.tensor(list(labels_train))
class_weights=class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y.numpy())
class_weights=torch.tensor(class_weights, dtype=torch.double)
loss_func = nn.BCEWithLogitsLoss(pos_weight=class_weights[1]/class_weights[0], reduction='mean') 
optimizer = optim.Adam(cnn.parameters(), lr = 0.0001)  

hyperparameters = (batch_size, d.item(), e.item(), c.item())

if not sd_path.exists():
    Path(statedict_path).mkdir(parents=True, exist_ok=True)
    auc_test, auc_train = train(cnn, batch_size, num_epochs, train_data, labels_train, loss_func, optimizer, (test_data, labels_test), draw=False, step_test=True, )
    cnn.auc_train = nn.Parameter(torch.tensor(auc_train), requires_grad=False)
    cnn.auc_test = nn.Parameter(torch.tensor(auc_test), requires_grad=False)
    torch.save(cnn.state_dict(), sd_path)
else:
    if cfg.retrain == 'True':
        auc_test, auc_train = train(cnn, batch_size, num_epochs, train_data, labels_train, loss_func, optimizer, (test_data, labels_test), draw=False, step_test=True, )
        cnn.auc_train = nn.Parameter(torch.tensor(auc_train), requires_grad=False)
        cnn.auc_test = nn.Parameter(torch.tensor(auc_test), requires_grad=False)
        torch.save(cnn.state_dict(), sd_path)
    elif cfg.retrain == 'False':
        sd = torch.load(sd_path)
        cnn.load_state_dict(sd, strict=True)
        cnn.double()
        cnn.eval()

In [54]:
resulting_hyperparameters = [param.data.item() for name, param in cnn.named_parameters()]

tp_test, tn_test, fp_test, fn_test, sigmas, results_test = test(cnn, test_data, labels_test)
test_stats = get_test_stats(test_data.shape[2], tp_test, tn_test, fp_test, fn_test, sigmas, print_res=False)

tp_train, tn_train, fp_train, fn_train, sigmas, results_train = test(cnn, train_data, labels_train)
train_stats = get_test_stats(train_data.shape[2], tp_train, tn_train, fp_train, fn_train, sigmas, print_res=False)

trains_stat_vals = list(vars(train_stats).values())
test_stat_vals = list(vars(test_stats).values())

export_stats = [(item_train, item_test) for (item_train, item_test) in zip(trains_stat_vals, test_stat_vals)]
network_params = NetworkParams(network_name, desc, filename, hyperparameters, resulting_hyperparameters, export_stats, cnn.auc_train, cnn.auc_test, comment)
network_params.save()

display_data = DisplayData(test_stats.conf_matrix, train_stats.conf_matrix)

In [55]:
image_data_mx, image_data_int = map_ids_to_color(train_id, test_id, results_train, results_test)
marked_events = map_events_to_color(events, train_id, test_id, results_train, results_test)
month_ticks = map_ids_to_color_by_day(train_id, test_id, results_train, results_test)
draw_colormesh_by_tick(image_data_mx, save=True, filename=network_name, display_data=display_data)
draw_colormesh_by_event(marked_events, save=True, filename=network_name, display_data=display_data)
draw_colormesh_by_month(month_ticks, save=True, filename=network_name, display_data=display_data)

In [56]:
tp_train_id, fp_train_id, fn_train_id, tn_train_id  = results_train
tp_test_id, fp_test_id, fn_test_id, tn_test_id  = results_test

In [57]:
tps_train = list(tp_train_id) + list(tn_train_id) 
tps_test = list(tp_test_id) + list(tn_test_id)
fps_train = list(fp_train_id) + list(fn_train_id) 
fps_test = list(fp_test_id) + list(fn_test_id)

tps = tps_train + tps_test
fps = fps_train + fps_test

'''
TP_vectors_test = np.empty(shape=[len(tps_test), 36 * 69])
FP_vectors_test = np.empty(shape=[len(fps_test), 36 * 69])

for i, id in enumerate(tps_test):
    img = data[:, :, id].flatten()
    TP_vectors_test[i] = img

for i, id in enumerate(fps_test):
    img = data[:, :, id].flatten()
    FP_vectors_test[i] = img
'''

TP_vectors_new = np.empty(shape=[len(tps), 36 * 69])
FP_vectors_new = np.empty(shape=[len(fps), 36 * 69])

for i, id in enumerate(tps):
    img = data[:, :, id].flatten()
    TP_vectors_new[i] = img

for i, id in enumerate(fps):
    img = data[:, :, id].flatten()
    FP_vectors_new[i] = img

In [58]:
np.save('../MatlabData/' + cfg.metric + '_TP_vectors_new.npy', TP_vectors_new)
np.save('../MatlabData/' + cfg.metric + '_FP_vectors_new.npy', FP_vectors_new)

In [48]:
from scipy.io import savemat

dic = {'TP_vectors_new' : TP_vectors_new,
       'FP_vectors_new' : FP_vectors_new }
savemat('../ErrorData.mat', dic)